In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using QPControl
using LCPSim
using MeshCat
using MeshCatMechanisms
using LearningMPC
using LearningMPC.Models
using RigidBodySim
using Gurobi
using SimpleQP
using AtlasRobot
using StaticArrays
using Rotations
using IKMimic

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [2]:
vis = Visualizer()
open(vis)

Listening on 0.0.0.0:8716...


INFO: Serving MeshCat visualizer at http://127.0.0.1:8716


In [3]:
Δt = 0.002

0.002

In [4]:
# state initialization
function init_atlas!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
        set_configuration!(state, findjoint(mechanism, "$(sideprefix)_arm_shx"), 
            sideprefix=='r' ? π/4 : -π/4)
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [0, 0.85, 0])
    state
end

init_atlas! (generic function with 1 method)

In [40]:
reduced_model = BoxAtlas(add_contacts=true)
position_bounds(findjoint(mechanism(reduced_model), "floating_base")) .= RigidBodyDynamics.Bounds(-Inf, Inf)
delete!(vis)
mvis_reduced = MechanismVisualizer(reduced_model, vis[:reduced])
copy!(mvis_reduced, nominal_state(reduced_model))
for bodyname in [
        "r_hand_mount",
        "l_hand_mount",
        "r_foot_sole",
        "l_foot_sole",
        "pelvis"
    ]
    setelement!(mvis_reduced, default_frame(findbody(mechanism(reduced_model), bodyname)))
end


full_model = AtlasRobot.mechanism()
for jointname in ["back_bkx", "back_bky", "back_bkz"]
    joint = findjoint(full_model, jointname)
    replace_joint!(full_model, joint, 
        Joint(jointname, frame_before(joint), frame_after(joint), Fixed{Float64}())
    )
end

joint = findjoint(full_model, "pelvis_to_world")
replace_joint!(full_model, joint,
    Joint("pelvis_to_world", frame_before(joint), frame_after(joint),
    Planar{Float64}(SVector(0., 1, 0), SVector(0., 0, 1)))
)
effort_bounds(findjoint(full_model, "pelvis_to_world")) .= RigidBodyDynamics.Bounds(0, 0)

remove_fixed_tree_joints!(full_model)
add_environment_primitive!(full_model, 
    HalfSpace3D(
        Point3D(root_frame(full_model), 0., 0, 0), 
        FreeVector3D(root_frame(full_model), 0., 0, 1)))

for (name, offset, orientation) in [
        ("r_foot_sole", (0.0426, 0.0, -0.07645), RotX(0.0)), 
        ("l_foot_sole", (0.0426, 0.0, -0.07645), RotX(0.0)),
        ("r_hand_mount", (0.0, -0.1, 0.0), RotZYX(0., 0., π/2)),
        ("l_hand_mount", (0.0, -0.1, 0.0), RotZYX(0., π, π/2))
    ]
    frame = CartesianFrame3D(name)
    inertia = SpatialInertia(frame, SDiagonal(0., 0, 0), SVector(0., 0, 0), 0.0)
    body = RigidBody(inertia)
    parent = findbody(full_model, name[1:6])
    joint = Joint(string(parent, "_to_", name), Fixed{Float64}())
    pose = Transform3D(frame_before(joint), default_frame(parent), orientation, SVector(offset...))
    attach!(full_model, parent, body, joint, joint_pose = pose)
end

mvis_full = MechanismVisualizer(full_model, Skeleton(), vis[:full])
copy!(mvis_full, init_atlas!(MechanismState(full_model)))
for bodyname in [
        "r_hand_mount",
        "l_hand_mount",
        "r_foot_sole",
        "l_foot_sole",
        "pelvis"
    ]
    setelement!(mvis_full, default_frame(findbody(full_model, bodyname)))
end

lqrsol = let params = MPCParams(reduced_model)
    params.Δt = 0.05
    LQRSolution(reduced_model, params)
end

(::LQRSolution) (generic function with 2 methods)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [56]:
mpc = QPControl.MPCController{QPControl.ContactPoint{4}}(
    mechanism(reduced_model),
    Gurobi.Optimizer(OutputFlag=0, TimeLimit=1)
)
copy!(mpc.state, nominal_state(reduced_model)) # ensure IKMimic has a good starting point
stage = addstage!(mpc, Δt)
floor = HalfSpace3D(Point3D(root_frame(mechanism(reduced_model)), 0., 0, 0),
    FreeVector3D(root_frame(mechanism(reduced_model)), 0., 0, 1))

for bodyname in ["r_foot_sole", "l_foot_sole"]
    body = findbody(mechanism(reduced_model), bodyname)
    position = Point3D(default_frame(body), 0., 0, 0)
    μ = 0.4
    contact = addcontact!(mpc, stage, position, floor, μ, 
        QPControl.LCPContact(separation_atol=1e-3),
#         QPControl.BooleanContact(separation_atol=1e-3)
    )
    contact.maxnormalforce[] = 1e6
end

objective = 0
for stage in mpc.stages
    x̄ = @expression vcat(stage.q, stage.v) - lqrsol.x0
    ū = @expression stage.u - lqrsol.u0
#     ū = stage.u
#     ū = stage.v̇
    objective = @expression objective + x̄' * lqrsol.Q * x̄ 
    objective = @expression objective + ū' * lqrsol.R * ū
    for contact in stage.contacts
        contact.weight[] = 1e-9
        objective = @expression objective + QPControl.objectiveterm(contact, mpc.qpmodel)
    end
end
x̄ = @expression vcat(last(mpc.stages).q, last(mpc.stages).v) - lqrsol.x0
objective = @expression objective + x̄' * lqrsol.S * x̄;
@objective mpc.qpmodel Minimize objective

momentum = MomentumBasedController{4}(full_model, Gurobi.Optimizer(OutputFlag=0),
    floatingjoint = findjoint(full_model, "pelvis_to_world")
)

for body in bodies(full_model)
    for point in RigidBodyDynamics.contact_points(body)
        position = location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(momentum, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6
        contact.weight[] = 1e-3
    end
end


task_map = Vector{Tuple{RigidBody{Float64}, QPControl.AbstractMotionTask}}()

for bodyname in ["pelvis", "r_hand_mount", "l_hand_mount"]
    body = findbody(full_model, bodyname)
    task = QPControl.PointAccelerationTask(
            full_model,
            path(full_model, root_body(full_model), body),
            Point3D(default_frame(body), 0., 0, 0)
            )
    addtask!(momentum, task)
    push!(task_map, (findbody(mechanism(reduced_model), bodyname), task))
end

oriented_bodies = [
    "pelvis",
    "r_foot_sole",
    "l_foot_sole"
]

orientation_control_tasks = Vector{AngularAccelerationTask}()

for bodyname in oriented_bodies
    body = findbody(full_model, bodyname)
    task = QPControl.AngularAccelerationTask(
            full_model,
            path(full_model, root_body(full_model), body)
            )
    addtask!(momentum, task)
    push!(orientation_control_tasks, task)
end

for joint in tree_joints(full_model)
    regularize!(momentum, joint, 1e-2)
end

xstar_full = init_atlas!(MechanismState(full_model))
joint_tasks = Vector{Tuple{JointID, Float64, JointAccelerationTask}}()
# for joint in tree_joints(full_model)
#     if contains(string(joint), "arm") || contains(string(joint), "back")
#         ref = configuration(xstar_full, joint)[]
#         id = JointID(joint)
#         task = JointAccelerationTask(joint)
#         addtask!(momentum, task, 1e-2)
#         push!(joint_tasks, (id, ref, task))
#     end
# end

matching_bodies = [
    "pelvis",
    "r_foot_sole",
    "l_foot_sole",
    "r_hand_mount",
    "l_hand_mount"
]
ik_workspace = IKMimic.IKMimicWorkspace(init_atlas!(MechanismState(full_model)), 
    nominal_state(reduced_model), matching_bodies, ["pelvis"])

mapped = LearningMPC.MappedController(
    mpc,
    momentum,
    task_map,
    orientation_control_tasks,
    joint_tasks,
    ik_workspace
)

highlevel = let mapped=mapped, mvis_full=mvis_full, mvis_reduced=mvis_reduced
    function (τ, t, x)
        copy!(mvis_full, x)
        try
            mapped(τ, t, x)
        finally
            set_configuration!(mvis_reduced, mapped.ik_workspace.x2[1:num_positions(mvis_reduced.state)])
    #         copy!(mvis_reduced, mapped.ik_workspace.x2)
        end
        sleep(0.01)
#         τ .= 0
    end
end

(::#51) (generic function with 1 method)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [57]:
state = init_atlas!(MechanismState(full_model))
# set_velocity!(state, findjoint(full_model, "pelvis_to_world"), 
#     [-0.1, 0, 0])
problem = LearningMPC.simulation_problem(state, highlevel, Δt, 2.0; damping_kd=1.0)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis_full, sol)

p̈ref = FreeVector3D in "world": [4.36873e-22, 7.13468e-6, -9.88534]
p̈ref = FreeVector3D in "world": [3.72086e-22, 42.9749, -52.3938]
p̈ref = FreeVector3D in "world": [3.72086e-22, -42.975, -52.3938]
p̈ref = FreeVector3D in "world": [-8.5644e-18, -0.139867, -41.8794]
p̈ref = FreeVector3D in "world": [-9.53957e-18, 30.357, -90.7864]
p̈ref = FreeVector3D in "world": [-9.53973e-18, -29.4675, -91.6489]
p̈ref = FreeVector3D in "world": [-4.76884e-17, -0.77881, -46.1087]
p̈ref = FreeVector3D in "world": [-5.44524e-17, 56.3422, -53.0616]
p̈ref = FreeVector3D in "world": [-5.44553e-17, -55.9688, -58.6217]
p̈ref = FreeVector3D in "world": [-2.03773e-16, -3.32787, -46.1206]
p̈ref = FreeVector3D in "world": [-2.34693e-16, 47.8922, -45.4601]
p̈ref = FreeVector3D in "world": [-2.34717e-16, -43.7053, -62.6569]
p̈ref = FreeVector3D in "world": [-8.48894e-16, -13.8635, -47.1912]
p̈ref = FreeVector3D in "world": [-9.77109e-16, 46.5408, -4.04631]
p̈ref = FreeVector3D in "world": [-9.77582e-16, -28.4808

true

p̈ref = FreeVector3D in "world": [1.56818e-14, 256.103, 107.447]
p̈ref = FreeVector3D in "world": [4.78613e-14, 69.1849, 99.462]
p̈ref = FreeVector3D in "world": [-1.98273e-14, 1.22622, -112.98]
p̈ref = FreeVector3D in "world": [1.59769e-14, 260.923, 110.339]
p̈ref = FreeVector3D in "world": [5.72513e-14, 125.591, 144.176]
p̈ref = FreeVector3D in "world": [-2.04787e-14, -0.172689, -113.605]


INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING


In [19]:
using Blink

In [21]:
body!(Window(), manipulate!(mvis_full))

Blink.Page(2, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(57) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#111),Pair{String,Any}("callback", Blink.#1)), Future(1, 1, 2, Nullable{Any}(true)))